In [4]:
from palantir.palantir import Palantir
from palantir.simulation import Simulation
from palantir.types import (
    Currency,
)
from palantir.util import (
    init_price_db,
)


def run_simulation():
    hours = 2000
    tokens = [
        Currency("bitcoin"),
        Currency("ethereum"),
        Currency("dai"),
    ]

    db = init_price_db(tokens, hours)

    def build_simulation():
        # TODO return a valid simulation object
        pass
    
    palantir = Palantir(
        simulation_factory=build_simulation,
        simulations_number=1,
    )
    
    # palantir.run()